In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

In [ ]:
data = pd.DataFrame({'value':[632, 1638, 569, 115, 433, 1130, 754, 555],
                     'patient':[1, 1, 1, 1, 2, 2, 2, 2],
                     'phylum':['Firmicutes', 'Proteobacteria', 'Actinobacteria', 
    'Bacteroidetes', 'Firmicutes', 'Proteobacteria', 'Actinobacteria', 'Bacteroidetes']})
data

In [ ]:
data[data['phylum'].apply(lambda x: x.endswith('bacteria')) & data['value'].apply(lambda x: x>1000)]

In [ ]:
len(data[data.phylum.isin(['Firmicutes', 'Actinobacteria'])])

Hierarchical index

In [ ]:
baseball = pd.read_csv("Data/baseball.csv", index_col='id')
baseball.head()

In [ ]:
baseball_h = baseball.set_index(['year', 'team', 'player'])

In [ ]:
baseball_h.index.is_unique

In [ ]:
baseball_h.loc[(2007, 'ATL', 'francju01')] #locating by tuples

In [ ]:
baseball[['player','sb','cs']].sort_values(ascending=[False,True], 
                                           by=['sb', 'cs']).head(10)

In [ ]:
baseball['obp']=baseball.apply(lambda p: (p.h+p.bb+p.hbp)/(p.ab+p.bb+p.hbp+p.sf) if (p.ab+p.bb+p.hbp+p.sf) != 0.0 else 0.0, axis=1)

In [ ]:
baseball.sum() # doesnt make sense for strings

In [ ]:
baseball.mean(numeric_only=True)

In [ ]:
baseball.player.describe()

In [ ]:
baseball.hr.corr(baseball.X2b)

Ships

In [ ]:
segments = pd.read_csv("Data/transit_segments.csv")
segments.head()

In [ ]:
vessels = pd.read_csv("Data/vessel_information.csv", index_col='mmsi')

In [ ]:
segments = pd.read_csv("Data/transit_segments.csv", parse_dates=['st_time', 'end_time'], nrows=1000)

In [ ]:
segments[segments.st_time.dt.month==2].head()

In [ ]:
pd.get_dummies(vessels.head(10).type)

Hi

In [3]:
cdystonia = pd.read_csv("Data/cdystonia.csv", index_col=None)
cdystonia.head()

,patient,obs,week,site,id,treat,age,sex,twstrs
0,1,1,0,1,1,5000U,65,F,32
1,1,2,2,1,1,5000U,65,F,30
2,1,3,4,1,1,5000U,65,F,24
3,1,4,8,1,1,5000U,65,F,37
4,1,5,12,1,1,5000U,65,F,39


In [4]:
twstrs_wide = cdystonia.pivot(index='patient', columns='obs', values='twstrs').head() # reshaping
twstrs_wide

obs,1,2,3,4,5,6
patient,,,,,,
1,32.0,30.0,24.0,37.0,39.0,36.0
2,60.0,26.0,27.0,41.0,65.0,67.0
3,44.0,20.0,23.0,26.0,35.0,35.0
4,53.0,61.0,64.0,62.0,NaN,NaN
5,53.0,35.0,48.0,49.0,41.0,51.0


In [5]:
cdystonia_wide = (cdystonia[['patient','site','id','treat','age','sex']]
                  .drop_duplicates()
                  .merge(twstrs_wide, right_index=True, left_on='patient', how='inner')
                  .head())
cdystonia_wide

,patient,site,id,treat,age,sex,1,2,3,4,5,6
0,1,1,1,5000U,65,F,32.0,30.0,24.0,37.0,39.0,36.0
6,2,1,2,10000U,70,F,60.0,26.0,27.0,41.0,65.0,67.0
12,3,1,3,5000U,64,F,44.0,20.0,23.0,26.0,35.0,35.0
18,4,1,4,Placebo,59,F,53.0,61.0,64.0,62.0,NaN,NaN
22,5,1,5,10000U,76,F,53.0,35.0,48.0,49.0,41.0,51.0


In [6]:
pd.melt(cdystonia_wide, id_vars=['patient','site','id','treat','age','sex'], var_name='obs', value_name='twsters').head()

,patient,site,id,treat,age,sex,obs,twsters
0,1,1,1,5000U,65,F,1,32.0
1,2,1,2,10000U,70,F,1,60.0
2,3,1,3,5000U,64,F,1,44.0
3,4,1,4,Placebo,59,F,1,53.0
4,5,1,5,10000U,76,F,1,53.0


In [7]:
pd.crosstab(cdystonia.sex, cdystonia.site)

site,1,2,3,4,5,6,7,8,9
sex,,,,,,,,,
F,52,53,42,30,22,54,66,48,28
M,18,29,30,18,11,33,6,58,33


In [8]:
cdystonia.treat.value_counts()

10000U     213
5000U      211
Placebo    207
Name: treat, dtype: int64

In [9]:
cdystonia['treatment'] = cdystonia.treat.map({'Placebo': 0, '5000U': 1, '10000U': 2})
cdystonia.treatment.head(10)

0    1
1    1
2    1
3    1
4    1
5    1
6    2
7    2
8    2
9    2
Name: treatment, dtype: int64

In [11]:
cdystonia.treatment.value_counts()

2    213
1    211
0    207
Name: treatment, dtype: int64

In [ ]:
pd.Categorical(cdystonia.treat)

In [15]:
cdystonia['treat'] = cdystonia.treat.astype('category')

In [16]:
cdystonia.treat.describe()

count        631
unique         3
top       10000U
freq         213
Name: treat, dtype: object

In [ ]:
cdystonia_grouped = cdystonia.groupby(cdystonia.patient)

In [ ]:
cdystonia_grouped.agg('mean').head()
cdystonia_grouped.mean().head() #equivalent 